In [1]:
import json
import h5py
from load_data import *
from model import *
from plotting import *
import tensorflow as tf 
import os
from numpy import sqrt as sqrt

Importamos los ground truth del conjunto de entrenamiento y test. 

In [2]:
json_train = open('JSON FILES\DTS_SG_part_A.json')
json_test = open('JSON FILES\DTS_SG_part_A_TEST.json')
paths_train = json.load(json_train)
paths_test = json.load(json_test)

#Cargamos la data 
train_GT = mLoad_GT(paths_train)
train_img = mLoad_Img(paths_train)

test_GT = mLoad_GT(paths_test)
test_img = mLoad_Img(paths_test)

In [3]:
f = h5py.File(r'C:\Users\Usuario\Documents\Nueva carpeta\Gaussian-Derivates-Network\training\resume.h5', 'r')

In [4]:
predict = np.asarray(f['predict'])

Construimos la red neuronal. Este sera un prototipo hasta siguiente cambio

Entrenamos un poco para testeo!

In [ ]:
pre_trained = r"training\cp-0050.ckpt"
model.load_weights(pre_trained)

In [6]:
count_estimate(test_img, test_GT, predict, model)